# DataSet #3: Mobilitat entre Municipis

Autor: Marc Bosch Manzano<br>
Data: Octubre 2025<br>
Iteració: 2a<br>
Projecte: MobiCat

## Descripció del Dataset

`movilidad_municipios_2023-01_origen.csv`

Descripció donada per Telefónica:

Este fichero contiene datos de movilidad entre los barrios de Barcelona y los 
municipios de la provincia de Barcelona. 
               
**1. day:** Día en el que se produce el viaje.<br>
**2. day_of_the_week:** Día de la semana en el que se produce el viaje (Lunes, Martes, etc.).<br>
**3. month:** Mes numerado del año (01 - 12).<br>
**4. origen:** Origen del visitante que viaja. Puede ser:

* (a) **Internacional:** Visitantes extranjeros.
* (b) **Nacional:** Residen fuera de la provincia de Barcelona.
* (c) **Regional:** Residen en la provincia de Barcelona excluyendo el municipio de Barcelona.
* (d) **Residente:** Residen en el municipio de Barcelona.

**5. municipio_origen:** Código del municipio donde empieza el viaje (Ej: 08019).<br>
**6. municipio_origen_name:** Nombre del municipio donde empieza el viaje (Ej: Barcelona).<br>
**7. barrio_origen:** Código del barrio donde empieza el viaje (Ej: 80190101).<br>
**8. barrio_origen_name:** Nombre del barrio donde empieza el viaje (Ej: el Raval).<br>
**9. municipio_destino:** Código del municipio donde termina el viaje (Ej: 08019).<br>
**10. municipio_destino_name:** Nombre del municipio donde termina el viaje (Ej: Barcelona).<br>
**11. barrio_destino:** Código del barrio donde termina el viaje (Ej: 80190103) (“-” para municipios distintos de Barcelona).<br>
**12. barrio_destino_name:** Nombre del barrio donde termina el viaje (Ej: la Barceloneta) (“-” para municipios distintos de Barcelona).<br>
**13. viajes:** Número total de viajes realizados. (Si el número de viajes es inferior a 15 se contará como 0).


## Exploració Bàsica del Dataset

### Importació de Llibreries

In [1]:
PACKAGES = ["pandas", "geopandas", "numpy", "seaborn", "matplotlib"]
COLORS = ["viridis", "plasma", "inferno", "magma", "cividis"]

import sys
sys.path.append('../mobicat_python_utils')

import utils # type: ignore
utils.install_missing_packages(PACKAGES)

import os
import seaborn as sns
import numpy as np
import pandas as pd 
import geopandas as gpd
import matplotlib.pyplot as plt

import IPython.display 
IPython.display.clear_output()

### Importació de les Dades de Testeig

**Inmportant**: importa al directori *test-data* els següents fitxers o canviar el directori pel directori amb els fitxers. 
```bash
    movilidad_barrios_2023-01_origen.csv
    movilidad_municipios_2023-01_origen.csv
    movilidad_mun_barrios_2023-01_origen.csv
```

In [2]:
directory = "./test-data/"
file_name = "movilidad_municipios_2023-01_origen.csv"
file_path = os.path.join(directory, file_name)

df = pd.read_csv(file_path, dtype=str)
df["viajes"] = df["viajes"].astype("int32")
df["day"] = pd.to_datetime(df["day"])

Imprimim unes fileres del nostre dataset i les seves dimensions.

In [14]:
display(df.sample(5))
print("Num rows:", df.shape[0])
print("Num columns:", df.shape[1])

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes
322724,2023-01-06,Viernes,01,Regional,08009,Argentona,08184,Rubí,15
287054,2023-01-05,Jueves,01,Regional,08139,Mura,08056,Castelldefels,0
1839619,2023-01-30,Lunes,01,Regional,08205,Sant Cugat del Vallès,08146,Olesa de Bonesvalls,15
819817,2023-01-13,Viernes,01,Regional,08905,"Palma de Cervelló, La",08164,"Pla del Penedès, El",0
1785357,2023-01-29,Domingo,01,Regional,08240,Sant Sadurní d'Anoia,08265,Sant Vicenç de Torelló,0


Num rows: 1921078
Num columns: 9


Imprimim els valors nuls del dataset i els tipus dels objectes. No hi ha valors nuls, així que suposarem que tots els datasets són correctes també.

In [4]:
df.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1921078 entries, 0 to 1921077
Data columns (total 9 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   day                     1921078 non-null  datetime64[ns]
 1   day_of_week             1921078 non-null  object        
 2   month                   1921078 non-null  object        
 3   origen                  1921078 non-null  object        
 4   municipio_origen        1921078 non-null  object        
 5   municipio_origen_name   1921078 non-null  object        
 6   municipio_destino       1921078 non-null  object        
 7   municipio_destino_name  1921078 non-null  object        
 8   viajes                  1921078 non-null  int32         
dtypes: datetime64[ns](1), int32(1), object(7)
memory usage: 124.6+ MB


Imprimim el nombre d'elements diferents per dataset.

In [5]:
df.nunique()

day                         31
day_of_week                  7
month                        1
origen                       4
municipio_origen           307
municipio_origen_name      307
municipio_destino          306
municipio_destino_name     306
viajes                    6656
dtype: int64

Visualització dels primers valors únics per atribut.

In [6]:
for column in df.columns:
    i = 0
    print(column, end=": ")
    uniques = df[column].unique()
    for value in uniques:
        print(value if column!="day" else str(value)[0:10], end="" if i == len(uniques) - 1 else "; ")
        if (i := i + 1) >= 8:
            print("..", end="")
            break
    print(".")
    

day: 2023-01-01; 2023-01-02; 2023-01-03; 2023-01-04; 2023-01-05; 2023-01-06; 2023-01-07; 2023-01-08; ...
day_of_week: Domingo; Lunes; Martes; Miércoles; Jueves; Viernes; Sábado.
month: 01.
origen: Regional; Internacional; Nacional; Residente.
municipio_origen: 08001; 08002; 08003; 08004; 08005; 08006; 08007; 08008; ...
municipio_origen_name: Abrera; Aguilar de Segarra; Alella; Alpens; Ametlla del Vallès, L'; Arenys de Mar; Arenys de Munt; Argençola; ...
municipio_destino: 08003; 08006; 08015; 08018; 08019; 08025; 08028; 08030; ...
municipio_destino_name: Alella; Arenys de Mar; Badalona; Balsareny; Barcelona; Bruc, El; Cabrera d'Anoia; Cabrils; ...
viajes: 0; 52; 324; 34; 87; 475; 95; 28; ...


### Filtres

Podem filtrar totes les files amb més de 0 viatges comptats. Això ens ajuda a estalviar computació en la resta de l'anàlisi.

In [16]:
df_zeros_filtered = df[df["viajes"] > 0]
current_rows = df_zeros_filtered.shape[0]
rows_reduced = df_zeros_filtered.shape[0] / df.shape[0]


display(df_zeros_filtered.sample(5))
print("Num curent rows: %d (%1.2f%% rows reduced)" % (current_rows, 100*rows_reduced))

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes
1742405,2023-01-29,Domingo,01,Regional,08016,Bagà,08011,Avià,16
525495,2023-01-09,Lunes,01,Regional,08113,Manresa,08187,Sabadell,717
1575526,2023-01-26,Jueves,01,Regional,08121,Mataró,08156,Palau-solità i Plegamans,86
925222,2023-01-15,Domingo,01,Internacional,08245,Santa Coloma de Gramenet,08019,Barcelona,482
1379649,2023-01-23,Lunes,01,Residente,08073,Cornellà de Llobregat,08184,Rubí,23


Num curent rows: 508839 (26.49% rows reduced)


Podem filtrar per dia de la setmana.

In [17]:
df_filter_1 = df_zeros_filtered[df_zeros_filtered["day_of_week"].isin(["Sábado", "Domingo"])]
df_filter_1.sample(5)

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes
821387,2023-01-14,Sábado,01,Regional,08007,Arenys de Munt,08230,Premià de Dalt,35
389583,2023-01-07,Sábado,01,Nacional,08035,Calella,08155,Palafolls,62
1680754,2023-01-28,Sábado,01,Regional,08010,Artés,08187,Sabadell,21
420327,2023-01-07,Sábado,01,Nacional,08187,Sabadell,08184,Rubí,53
436815,2023-01-07,Sábado,01,Regional,08270,Sitges,08263,Sant Vicenç dels Horts,83


Podem filtrar per municipi d'origen i mostrar cinc mostres aleatòries ordendadament.

In [18]:
df_filter_2 = df_zeros_filtered[df_zeros_filtered["municipio_origen_name"] == "Granollers"]
df_filter_2.sample(5).sort_values(by="viajes", ascending=False)

,day,day_of_week,month,origen,municipio_origen,municipio_origen_name,municipio_destino,municipio_destino_name,viajes
276177,2023-01-05,Jueves,01,Regional,08096,Granollers,08033,Caldes de Montbui,936
520490,2023-01-09,Lunes,01,Regional,08096,Granollers,08035,Calella,64
18953,2023-01-01,Domingo,01,Internacional,08096,Granollers,08260,Santa Perpètua de Mogoda,40
1266508,2023-01-21,Sábado,01,Residente,08096,Granollers,08205,Sant Cugat del Vallès,36
1384195,2023-01-23,Lunes,01,Regional,08096,Granollers,08208,Sant Esteve Sesrovires,34


### Agrupaments

Podem agrupar per la suma de viatges per municipi d'origen. Ordenem per viatges 10 municipis escollits aleatòriament.

In [43]:
df_grouped_1 = df_zeros_filtered.groupby(by="municipio_origen_name", as_index=False)["viajes"].sum()
df_grouped_1.sort_values(by="viajes", ascending=False).head(10)

,municipio_origen_name,viajes
20,Barcelona,21038046
103,"Hospitalet de Llobregat, L'",5366782
186,Sabadell,3507736
14,Badalona,3486870
170,"Prat de Llobregat, El",3038235
201,Sant Cugat del Vallès,2925947
270,Terrassa,2901657
74,Cornellà de Llobregat,2858300
125,Mataró,1998627
195,Sant Boi de Llobregat,1735898


Podem agrupar per la suma de viatges d'un dia concret. Ordenem pel dia.

In [44]:
df_grouped_2 = df_zeros_filtered.groupby(by="day", as_index=False)["viajes"].sum()
df_grouped_2.head(10).sort_values(by="day", ascending=True)

,day,viajes
0,2023-01-01,2169439
1,2023-01-02,3465322
2,2023-01-03,3671647
3,2023-01-04,3917849
4,2023-01-05,3600172
5,2023-01-06,2421411
6,2023-01-07,3060666
7,2023-01-08,2228300
8,2023-01-09,3915990
9,2023-01-10,4069559


També ho podem fer per setmana, ordenant per nombre de viatges.

In [45]:
df_grouped_2 = df_zeros_filtered.groupby(by="day_of_week", as_index=False)["viajes"].sum()
df_grouped_2.sort_values(by="viajes", ascending=False)

,day_of_week,viajes
3,Martes,18811868
2,Lunes,18600579
4,Miércoles,15951547
1,Jueves,15635250
6,Viernes,14677035
5,Sábado,12160058
0,Domingo,11220581


Finalment, podem agrupar tots els viatges del mes.

In [46]:
df_grouped_2 = df_zeros_filtered.groupby(by="month", as_index=False)["viajes"].sum()
df_grouped_2

,month,viajes
0,01,107056918
